In [1]:
#importing useful packages
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from astropy.io import ascii
import astropy.time
from astropy.time import Time
import astropy.table
from astropy.table import Table, Column, MaskedColumn
from astroplan import Observer
from astroquery.jplhorizons import Horizons
import pandas as pd
import csv
import astropy.time
import dateutil.parser

### Reading CSV file of dates targets were observed
raw_target_info = pd.read_csv("look_table.csv")

In [2]:
### Creating new dataframe with first date each target was observed
subset_df = raw_target_info[["Target","Date"]]
grouped_and_first_df = subset_df.groupby(["Target"]).first()
new_df = grouped_and_first_df.reset_index()
new_df.columns = ["Name","N"]
new_df.tail(5)

,Name,N
13,C/2021 A6,21/01/2021
14,C/2021 A7,21/01/2021
15,C/2021 B3,13/02/2021
16,P/2020 V3,07/12/2020
17,P/2020 V4,10/12/2020


In [3]:
### 
target_location = '474' # Mt. John Observatory MPC code

obsDate_lst = []
for i in new_df.N:
    obsDate_lst.append(i)

print(len(obsDate_lst))

targetname_lst = []
for i in new_df.Name:
    targetname_lst.append(i)

print(len(targetname_lst))

# need to convert epoch date to julian date
jd_obsDate_lst = []
for i in obsDate_lst:
    dt = dateutil.parser.parse(obsDate_lst[0]) 
    time = astropy.time.Time(dt)
    jd_obsDate_lst.append(time.jd)

print(len(jd_obsDate_lst))

18
18
18


In [4]:
# #need to query information from JPL Horizons database
eph_lst = []
el_lst = []
obj_lst = []
for index,jdDate in enumerate(jd_obsDate_lst):
    if targetname_lst[index] == "C/2018 F4":
        obj = Horizons(id='90004395', epochs=jdDate)
        eph_lst.append(obj.ephemerides()) #from eph, only need 'r' heliocentric distance
        el_lst.append(obj.elements()) #from elems, need a0 (semimajor axis), q, and T (perihelion date)
    else:
        obj = Horizons(id=targetname_lst[index], epochs=jdDate)
        obj_lst.append(obj)
        eph_lst.append(obj.ephemerides()) #from eph, only need 'r' heliocentric distance
        el_lst.append(obj.elements()) #from elems, need a0 (semimajor axis), q, and T (perihelion date)

In [5]:
print(len(eph_lst))
print(len(el_lst))

18
18


In [6]:
final_array = []
for index,target in enumerate(eph_lst):
    final_array.append([el_lst[index]['a'][0],eph_lst[index]['r'][0],
                        el_lst[index]['q'][0],el_lst[index]['Tp_jd'][0]])

In [7]:
eph_el_df = pd.DataFrame(final_array,columns=['a','r','q','T'])
eph_el_df.tail(5)

,a,r,q,T
13,-3.115937e-06,7.954310,4.928464,2.459550e+06
14,-3.258038e-06,2.855258,1.965796,2.459343e+06
15,-8.568433e-07,2.229828,1.878752,2.459258e+06
16,-6.356858e-06,6.230639,2.926078,2.458839e+06
17,-7.393996e-06,5.226928,1.507296,2.458846e+06


In [8]:
final_new_df = pd.concat([new_df,eph_el_df],axis=1)
final_new_df.tail(5)

,Name,N,a,r,q,T
13,C/2021 A6,21/01/2021,-3.115937e-06,7.954310,4.928464,2.459550e+06
14,C/2021 A7,21/01/2021,-3.258038e-06,2.855258,1.965796,2.459343e+06
15,C/2021 B3,13/02/2021,-8.568433e-07,2.229828,1.878752,2.459258e+06
16,P/2020 V3,07/12/2020,-6.356858e-06,6.230639,2.926078,2.458839e+06
17,P/2020 V4,10/12/2020,-7.393996e-06,5.226928,1.507296,2.458846e+06


In [9]:
final_new_df = final_new_df[['Name','a','N','r','q','T']]
final_new_df.tail(5)

,Name,a,N,r,q,T
13,C/2021 A6,-3.115937e-06,21/01/2021,7.954310,4.928464,2.459550e+06
14,C/2021 A7,-3.258038e-06,21/01/2021,2.855258,1.965796,2.459343e+06
15,C/2021 B3,-8.568433e-07,13/02/2021,2.229828,1.878752,2.459258e+06
16,P/2020 V3,-6.356858e-06,07/12/2020,6.230639,2.926078,2.458839e+06
17,P/2020 V4,-7.393996e-06,10/12/2020,5.226928,1.507296,2.458846e+06


In [10]:
yeet = final_new_df['T'][0]
print(yeet)
t = Time(final_new_df['T'][0], format='jd').to_value('iso')[:10]
print(t)
# t = Time(times, format='isot', scale='utc')

2459240.592245275
2021-01-26


In [11]:
final_new_df['T'] = final_new_df['T'].map(lambda name: Time(name, format='jd').to_value('iso')[:10])

# for i in final_new_df['T']:
#     print(i)
# #     i = Time(i, format='jd').to_value('iso')[:10]

In [13]:
#now we need to make the JPL data and the FITS data into tables
final_r_lst = []
for target in eph_lst:
    final_r_lst.append(target['r'][0])

In [15]:
final_a0_lst = []
final_q_lst = []
final_T_lst = []
for target in el_lst:
    final_a0_lst.append(target['a'][0]) 
    final_q_lst.append(target['q'][0])
    final_T_lst.append(target['Tp_jd'][0])